In [2]:
import os
import copy
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import glob
import numpy as np
import tensorflow as tf
from skimage.io import imread, imsave
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn import svm
from PIL import Image

In [3]:
src = '/Users/roryliao/Desktop/kvasir-dataset/Images'
train_dir = '/Users/roryliao/Desktop/kvasir-dataset/Train'
test_dir = '/Users/roryliao/Desktop/kvasir-dataset/Test'
train_padded_dir = '/Users/roryliao/Desktop/kvasir-dataset/Train_Padded'
test_padded_dir = '/Users/roryliao/Desktop/kvasir-dataset/Test_Padded'

In [4]:
train_datagen = ImageDataGenerator(width_shift_range = 0.2, height_shift_range = 0.2, horizontal_flip = True, vertical_flip = True, rescale=1/255, rotation_range = 90, validation_split = 0.1)
#Look up documentation. Look at creating new data using rotations
test_datagen = ImageDataGenerator(rescale=1/255)

In [5]:
train_gen = train_datagen.flow_from_directory(train_dir, batch_size = 2, seed = 42, subset = 'training', target_size = (256, 256))
val_gen = train_datagen.flow_from_directory(train_dir, batch_size = 2, seed = 42, subset = 'validation', target_size = (256, 256))
test_gen = test_datagen.flow_from_directory(test_dir, batch_size = 2, shuffle = False, target_size = (256, 256))

Found 2883 images belonging to 8 classes.
Found 317 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [12]:
pool_size = (2,2)
kernal_size = (3,3)
#Look into Regularization
#Decrease number of neurons in the Dense Layers
#Look into Auto Encoders
#Benchmark of 75%
#Adding more dense layers

x_in = tf.keras.layers.Input(shape=(256, 256, 3))

x = tf.keras.layers.Conv2D(filters=8, kernel_size=kernal_size, activation='relu')(x_in)
x = tf.keras.layers.MaxPool2D(pool_size=pool_size, strides = 2)(x)

x_flatten = tf.keras.layers.Flatten()(x)
x_flatten = tf.keras.layers.Flatten()(x_flatten)

x = tf.keras.layers.Dense(32, activation='relu')(x_flatten)
#Look into adding a Dense layer here

x_out = tf.keras.layers.Dense(8, activation='softmax')(x)

In [13]:
model = tf.keras.Model(x_in, x_out)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 254, 254, 8)       224       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 127, 8)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 129032)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 129032)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4129056   
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 264 

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])

In [15]:
history = model.fit_generator(train_gen, steps_per_epoch=len(train_gen), epochs=10, validation_data=val_gen, validation_steps=len(val_gen))

Epoch 1/10
1442/1442 [==============================] - 129s 90ms/step - loss: 2.1869 - acc: 0.1148 - val_loss: 2.0794 - val_acc: 0.1293
Epoch 2/10
1442/1442 [==============================] - 116s 81ms/step - loss: 2.0802 - acc: 0.1224 - val_loss: 2.0794 - val_acc: 0.1293
Epoch 3/10
1442/1442 [==============================] - 124s 86ms/step - loss: 2.0826 - acc: 0.1183 - val_loss: 2.0794 - val_acc: 0.1293
Epoch 4/10
1442/1442 [==============================] - 122s 85ms/step - loss: 2.0801 - acc: 0.1283 - val_loss: 2.0794 - val_acc: 0.1293
Epoch 5/10
1442/1442 [==============================] - 119s 83ms/step - loss: 2.0802 - acc: 0.1245 - val_loss: 2.0794 - val_acc: 0.1293
Epoch 6/10
1442/1442 [==============================] - 135s 94ms/step - loss: 2.0801 - acc: 0.1179 - val_loss: 2.0794 - val_acc: 0.1293
Epoch 7/10
1442/1442 [==============================] - 121s 84ms/step - loss: 2.0802 - acc: 0.1221 - val_loss: 2.0794 - val_acc: 0.1293
Epoch 8/10
1442/1442 [===================

In [11]:
def show_final_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[0].set(xlim=(0, 9), ylim=(1, 2.5))
    ax[0].set_xlabel('Epochs')
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["acc"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_acc"], label="Validation acc")
    ax[1].set(xlim=(0, 9), ylim=(0, 1))
    ax[1].set_xlabel('Epochs')
    ax[0].legend()
    ax[1].legend()

In [ ]:
show_final_history(history)